In [1]:
import os
import time

import numpy as np
import torch

import argparse

import torch.onnx

from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer, 
                          Phi3ForCausalLM,
                          Phi3Config,
                          pipeline,
                          )
from transformers.pipelines.pt_utils import KeyDataset
from huggingface_hub import hf_hub_download
import torch_tensorrt 
import utils
import torch.backends.cudnn as cudnn
device = torch.device( "cuda" if torch.cuda.is_available() else cpu )

In [2]:
parser = argparse.ArgumentParser(description="TensorRT")
parser.add_argument("--model_path", type=str, default="models/")
parser.add_argument("--model_name", type=str, default="Phi-3-medium-4k-instruct")
parser.add_argument("--data_path", type=str, default="data/")
parser.add_argument("--dataset", type=str, default="test_dataset.jsonl")
parser.add_argument('--seed',type=int, default=0)
parser.add_argument("--batch_size", type=int, default=1)
parser.add_argument('--dtype',type=str, default="auto")
parser.add_argument('--n', type=int, default=1)
parser.add_argument('--temperature', type=float, default=0.0)

config = parser.parse_args([])

utils.seed_everything(config.seed)
CURR_PATH = os.getcwd()

In [3]:
# 1. 모델 및 토크나이저 로드
model_id = os.path.join(CURR_PATH, config.model_path, config.model_name)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

max_memory = {
    0: "10GB",  # GPU 0 
    'cpu': "20GB"  # CPU 
}

llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    max_memory = max_memory 
)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
# 데이터 로드
data = load_dataset("json", data_files="./data/test_dataset.jsonl")['train']
messages = KeyDataset(data, 'message')

# 시스템 프롬프트 정의
sys = "You are a helpful AI Assistant. Do not hallucinate."

# 메시지 준비
messages = list(messages)
token_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
token_ids.insert(0, "<|system|>" + sys + "<|end|>")

# 입력 텐서로 변환 및 데이터 타입 변경
inputs = tokenizer(token_ids, return_tensors="pt", padding=True).to("cuda")

In [7]:
generation_args = {
    "max_new_tokens": 1024,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [6]:
torch.onnx.export(llm,               # 실행될 모델
                    inputs['input_ids'],                         # 모델 입력값 (튜플 또는 여러 입력값들도 가능)
                    "./models/lightweight_phi3.onnx",   # 모델 저장 경로 (파일 또는 파일과 유사한 객체 모두 가능)
                    export_params=True,        # 모델 파일 안에 학습된 모델 가중치를 저장할지의 여부
                    opset_version=10,          # 모델을 변환할 때 사용할 ONNX 버전
                    do_constant_folding=True,  # 최적하시 상수폴딩을 사용할지의 여부
                    input_names = ['inputs'],   # 모델의 입력값을 가리키는 이름
                    output_names = ['outputs'], # 모델의 출력값을 가리키는 이름
                    dynamic_axes={'inputs' : {0 : 'batch_size', 1 :'sequence'},    # 가변적인 길이를 가진 차원, 없으면 None
                                'outputs' : {0 : 'batch_size', 1:'output'}})

  if input_shape[-1] > 1 or self.sliding_window is not None:

  if past_key_values_length > 0:

  if attn_weights.size() != (bsz, self.num_heads, q_len, kv_seq_len):

  if attention_mask.size() != (bsz, 1, q_len, kv_seq_len):

  if attn_output.size() != (bsz, self.num_heads, q_len, self.head_dim):



RuntimeError: NVML_SUCCESS == r INTERNAL ASSERT FAILED at "../c10/cuda/CUDACachingAllocator.cpp":838, please report a bug to PyTorch. 

In [16]:
trt_model = torch.compile(llm, backend="torch_tensorrt", dynamic=False)

In [ ]:
import onnx

onnx_model = onnx.load("super_resolution.onnx")
onnx.checker.check_model(onnx_model)

In [17]:
torch.cuda.empty_cache() #cache를 지움
generation_args = {
    "max_new_tokens": 1024,  
    "temperature": 0.0,
    # "top_p": 0.95,
    "do_sample": False,
}

In [18]:
# torch.cuda.synchronize()

# 추론 시작 시간 측정
start = time.perf_counter()

# 데이터 로드
data = load_dataset("json", data_files="./data/test_dataset.jsonl")['train']
messages = KeyDataset(data, 'message')

# 시스템 프롬프트 정의
sys = "You are a helpful AI Assistant. Do not hallucinate."

# 메시지 준비
messages = list(messages)
token_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
token_ids.insert(0, "<|system|>" + sys + "<|end|>")

# 입력 텐서로 변환 및 데이터 타입 변경
inputs = tokenizer(token_ids, return_tensors="pt", padding=True).to("cuda")
# shape => torch.Size([32, 868])

# 추론 실행
with torch.no_grad():
    outs = trt_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs.get("attention_mask"),
        **generation_args
    )
# 생성된 토큰을 텍스트로 디코딩
generated_texts = tokenizer.batch_decode(outs, skip_special_tokens=True)
processed_outs = []
for idx, text in enumerate(generated_texts):
    if idx == 0:
        continue  # 첫 번째 출력은 건너뜀
    # 마지막 \n 이후의 부분만 추출
    last_newline_pos = text.rfind('\n')
    if last_newline_pos != -1:
        extracted_text = text[last_newline_pos + 1:].strip()
    else:
        extracted_text = text.strip()
    # 결과를 딕셔너리로 감싸고 리스트에 추가
    processed_outs.append([{"generated_text": extracted_text}])

# torch.cuda.synchronize()

# 추론 종료 시간 측정
end = time.perf_counter()

Generating train split: 0 examples [00:00, ? examples/s]

  warnings.warn(



In [19]:
print("===== Answers =====")
correct = 0
for i, out in enumerate(processed_outs):
    correct_answer = data[i]["answer"]
    answer = out[0]["generated_text"].lstrip().replace("\n","")
    if answer == correct_answer:
        correct += 1
    # print(answer)
 
print("===== Perf result =====")
print("Elapsed_time: ", end-start)
print(f"Correctness: {correct}/{len(data)}")

===== Answers =====
===== Perf result =====
Elapsed_time:  52.5368714556098
Correctness: 23/31
